# Title goes here
## Introduction goes here
Intro body goes here

In [1]:
import requests
import pandas as pd
import re
import time
import numpy as np
from bs4 import BeautifulSoup
import usaddress
from sklearn.model_selection import train_test_split
# from seleniumwire import webdriver
# from seleniumwire.webdriver.chrome.options import Options

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
def parse_num_result(input_string):
    try:
        # remove ',' in numerical result
        input_string = input_string.replace(',', '')
        regex_num_result = re.compile('(\d+)')
        # should be a list of all numerical values in the input_string
        num_result_list = regex_num_result.search(input_string)
        return int(num_result_list.group())
    except AttributeError:
        return None

def parse_summary(input_string):
    num_bed = None
    num_bath = None
    area = None
    
    input_string = input_string.replace(',', '')
    regex_summary = re.compile('([\d\.]*).*bd([\d\.]*).*ba([\d\.]*).*sqft')
    summary_list = regex_summary.search(input_string)
    if summary_list.group(1) != '':
        num_bed = int(summary_list.group(1))
    if summary_list.group(2) != '':
        num_bath = float(summary_list.group(2))
    if summary_list.group(3) != '':
        area = int(summary_list.group(3))
    return num_bed, num_bath, area 

In [ ]:
# url
# https://www.zillow.com/pittsburgh-pa/rentals/
# https://www.zillow.com/pittsburgh-pa/rentals/2_p/
# 40 results per page

def get_url(page_num):
    base = 'https://www.zillow.com/pittsburgh-pa/rentals/'
    if page_num == 1:
        return base
    else:
        return base + str(page_num) + '_p/'

def parse_scores(address):
    url = 'https://www.walkscore.com/score/' + address
    response = requests.get(url)
    root = BeautifulSoup(response.content, 'html.parser')
    
    regex_score = re.compile('(\d+).svg')
    
    walk_score = None
    tran_score = None
    
    walk_score_content = root.find('div', {'class': 'block-header-badge score-info-link', 'data-eventsrc': 'score page walk badge'})
    tran_score_content = root.find('div', {'class': 'block-header-badge score-info-link', 'data-eventsrc': 'score page transit badge'})
    if walk_score_content != None:
        walk_score = int(regex_score.search(str(walk_score_content)).group(1))
    if tran_score_content != None:
        tran_score = int(regex_score.search(str(tran_score_content)).group(1))
    return walk_score, tran_score
        
def parse_zillow_search_page(url):
    headers = {
       'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
       'accept-encoding': 'gzip, deflate, br',
       'accept-language': 'en-US,en;q=0.8',
       'upgrade-insecure-requests': '1',
       'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    root = BeautifulSoup(response.content, 'html.parser')
#     print(root)
#     num_result = parse_num_result(root.find('span', {'class':'result-count'}).get_text())
    
    df = pd.DataFrame(columns=['Price', 'Address', 'PropertyType', 'Beds', 'Baths', 'Sqft', 'YearBuilt', 'WalkScore', 'TransitScore',
                               'ParkingPrice', 'ParkingType', 'Cooling', 'Laundry', 'Description'])
    
    df = df.astype({'Price': 'float64', 'PropertyType': 'int64', 'Beds': 'int64', 'Baths': 'float64', 'Sqft': 'float64',
                   'YearBuilt': 'int64', 'WalkScore': 'int64', 'TransitScore': 'int64', 'ParkingPrice': 'int64', 'ParkingType': 'int64', 'Cooling': 'bool',
                   'Laundry': 'int64'})
    
    for div in root.find_all('div', {'class': 'list-card-info'}):
        for a in div.find_all('a', {'class': 'list-card-link list-card-link-top-margin'}):
            sub_url = a['href']
            time.sleep(0.1)
            results = None
            # apartment with special listing
            if sub_url.startswith('/b/'):
                sub_url = 'https://www.zillow.com' + sub_url
                results = parse_zillow_apartment_page(sub_url)
            # house
            else:
                results = parse_zillow_house_page(sub_url)
            if results != None:
                for result in results:
                    df = df.append(pd.Series(result, index=df.columns), ignore_index=True)
#     print(df)
    return df
    
                
                
# https://www.zillow.com/b/401-s-aiken-ave-pittsburgh-pa-9PRKLw/
def parse_zillow_apartment_page(url):
    if url == None or url == '':
        return None
    print(url)
    headers = {
       'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
       'accept-encoding': 'gzip, deflate, br',
       'accept-language': 'en-US,en;q=0.8',
       'upgrade-insecure-requests': '1',
       'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    root = BeautifulSoup(response.content, 'html.parser')
    
    low_prices = []
    high_prices = []
    areas = []
    num_beds = []
    num_baths = []
    
    units = root.find('div', {'id': 'building-floorplan-card-group'})
    if units == None:
        units = root.find('div', {'data-test-id': 'BuildingUnitsCardGroup'})
    if units == None:
        return None
    for div in units.children:
#         if not div.has_attr('style'):
        price_text = div.find('div', {'class': 'units-table__text--sectionheading'})
        if price_text == None:
            price_text = div.find('span', {'class': 'unit-card__unit-price units-table__text--sectionheading'})
        price_text = price_text.get_text()
        if '-' in price_text:
            low_price = parse_num_result(price_text.split('-')[0])
            high_price = parse_num_result(price_text.split('-')[1])
        else:
            price = parse_num_result(price_text)
            low_price = price
            high_price = price
        low_prices.append(low_price)
        high_prices.append(high_price)

        for index, span in enumerate(div.find_all('span', {'class': 'units-table__text--smallbody bdp-home-dna-val'})):
            if index == 0 and span.get_text() == 'Studio': 
                num_beds.append(0)
            elif index == 0:
                if span.get_text() == '--':
                    num_beds.append(None)
                else:
                    num_beds.append(int(span.get_text()))
            elif index == 1: 
                if span.get_text() == '--':
                    num_baths.append(None)
                else:
                    num_baths.append(float(span.get_text()))
            elif index == 2: 
                if span.get_text() == '--':
                    areas.append(None)
                else:
                    areas.append(float(parse_num_result(span.get_text())))
    cooling = None
    parking_type = None
    laundry = None
    parking_price = None
    walk_score = None
    tran_score = None
    address_st = None
    year_built = None
    description = None
    property_type = 0
    
    try:
        description = root.find('div', {'data-test-id': 'bdp-description'}).get_text()
    except AttributeError:
        description = None
    
    address = root.find('h2', {'data-test-id': 'bdp-building-address'})
    if address == None:
        return None
    address = address.get_text()
    if 'Undisclosed' in address:
        return None
    regex_address_st = re.compile('(.+)(Ave|Rd|Blvd|St|Dr|Sq|Way|Ct|Pl|Ln|Ctr|Cir|Ter)\s?(N|S|E|W)?(?:.*?)(,[\w\s]+)(,.+)')
    address = regex_address_st.search(address)
    if address == None:
        address = root.find('h2', {'data-test-id': 'bdp-building-address'}).get_text()
        address = address.replace('#xA0', ' ')
        address = regex_address_st.search(address)
    if address == None:
        return None   
    address_st = str(''.join([x for x in address.groups() if x != None]))
    walk_score, tran_score = parse_scores(address_st)
    
    # check facts top summary
    facts_top = root.find('div', {'id': 'bdp-building-amenities-summary'})
    if facts_top != None:
        for div in facts_top.find_all('div', {'class': 'Flex-c11n-8-29-0__n94bjd-0 iQllGd'}):
            if 'Year Built' in div.get_text():
                year_built = parse_num_result(div.get_text())
            
        
    
    
    for li in root.find_all('li', {'class': 'Flex-c11n-8-29-0__n94bjd-0 eegotE'}):
        info_text = li.get_text().lower()
        if 'cooling' in info_text:
            cooling = True
        elif 'parking' in info_text:
            parking_type  = 1
            if 'covered parking' in info_text:
                parking_price = 0

    # check building amenities
    building_amenities = root.find('div', {'data-test-id': 'Building amenities', 'class': 'sc-irlOZD fAFzrt'})
    if building_amenities != None:
        building_amenities_text = building_amenities.get_text().lower()
        if 'parking' in building_amenities_text or 'garage' in building_amenities_text:
            parking_type = 1
        if 'laundry' in building_amenities_text or 'washer' in building_amenities_text or 'dryer' in building_amenities_text:
            laundry = 1
            
    # check unit amenities
    unit_amenities = root.find('div', {'data-test-id': 'Unit features', 'class': 'sc-irlOZD fAFzrt'})
    if unit_amenities != None:
        unit_amenities_text = unit_amenities.get_text().lower()
        if 'washer' in unit_amenities_text or 'dryer' in unit_amenities_text:
            laundry = 2
                
    policies = root.find('div', {'data-test-id': 'building-policies'})
    if policies != None and parking_price == None:
        parking_policies = policies.find('div', {'data-test-id': 'building-policy-parking'})
        if parking_policies != None:
            parking_policies_text = parking_policies.get_text()    
            regex_parking_price = re.compile('\$(\d+)')
            parking_price_lists = regex_parking_price.search(parking_policies_text)
            if parking_price_lists != None:
                parking_price = min([parse_num_result(x) for x in parking_price_lists.groups()])
    
    common_result = [year_built, walk_score, tran_score, 
              parking_price, parking_type, cooling, laundry, description]
    result = []
    for i in range(len(low_prices)):
        result.append([low_prices[i], address_st, property_type, num_beds[i], num_baths[i], areas[i]] + common_result)
    return result
    
    
            
# https://www.zillow.com/homedetails/151-Robinson-St-38-Pittsburgh-PA-15213/2071373848_zpid/
def parse_zillow_house_page(url):
    if url == None or url == '':
        return None
    print(url)
    headers = {
       'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
       'accept-encoding': 'gzip, deflate, br',
       'accept-language': 'en-US,en;q=0.8',
       'upgrade-insecure-requests': '1',
       'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36'
    }
    response = requests.get(url, headers=headers)
    root = BeautifulSoup(response.content, 'html.parser')
    try:
        price = parse_num_result(root.find('span', {'class': 'Text-c11n-8-33-0__aiai24-0 sc-pkUbs dbPIZT'}).get_text())
        low_price = price
        high_price = price
    except AttributeError:
        price = None
    
    try:
        address = root.find('title').get_text()
        if 'Undisclosed' in address:
            return None
        address = address.replace('| Zillow', '')
        regex_address_st = re.compile('(.+)(Ave|Rd|Blvd|St|Dr|Sq|Way|Ct|Pl|Ln|Ctr|Cir|Ter)\s?(N|S|E|W)?(?:.*?)(,[\w\s]+)(,.+)')
        address = regex_address_st.search(address)
        if address == None:
            address = root.find('h2', {'class': 'Text-c11n-8-33-0__aiai24-0 eAgJuz ds-price-change-address-row'}).get_text()
            address = address.replace('#xA0', ' ')
            address = regex_address_st.search(address)
        if address == None:
            return None
        address_st = str(''.join([x for x in address.groups() if x != None]))
    except AttributeError:
        
        return None
    
    try:
        property_icon = root.find('i', {'class': 'sc-qPLKk fonsds zsg-icon-buildings'})
        property_type = property_icon.find_next_sibling('span', {'class': 'Text-c11n-8-33-0__aiai24-0 sc-pAyMl eWnzUA'}).get_text()
        property_type = 0 if property_type == 'Apartment' or property_type == 'Studio' else 1 if property_type == 'SingleFamily' or property_type == 'Townhouse' else 2
    except AttributeError:
        property_type = None
        
    try:
        summary_row = root.find('div', {'class': 'ds-summary-row'}).find('div', {'class': 'ds-bed-bath-living-area-header'}).get_text()
        num_bed, num_bath, area = parse_summary(summary_row)
    except AttributeError:
        num_bed = None
        num_bath = None
        area = None
        
    try:
        year_built = None
        condition = root.find('h6', {'class': 'Text-c11n-8-33-0__aiai24-0 flSprY'}, text='Condition')
        condition_detail = condition.find_next_sibling()
        for span in condition_detail.find_all('span', {'class': 'Text-c11n-8-33-0__aiai24-0 kTUCqv'}):
            if 'Year built' in span.get_text():
                year_built = parse_num_result(span.get_text())
    except AttributeError:
        year_built = None
        
    try:
        parking_price = None
        parking_type = None
        parking_icon = root.find('i', {'class': 'sc-qPLKk fonsds zsg-icon-parking'})
        parking_type = parking_icon.find_next_sibling('span', {'class': 'Text-c11n-8-33-0__aiai24-0 sc-pAyMl eWnzUA'}).get_text() 
        parking_price = 0 if 'Garage' in parking_type and property_type == 1 else 0 if 'Off' in parking_type else None
        parking_type = 0 if 'None' in parking_type else 1  
    except AttributeError:
        parking_type = None
        parking_price = None
    
    try:
        cooling = None
        cooling_icon = root.find('i', {'class': 'sc-qPLKk fonsds zsg-icon-snowflake'})
        if cooling_icon != None:
            cooling_type = cooling_icon.find_next_sibling('span', {'class': 'Text-c11n-8-33-0__aiai24-0 sc-pAyMl eWnzUA'}).get_text()
            if cooling_type != None and cooling_type != 'None':
                cooling = True
            elif cooling_type == 'None':
                cooling = False
    except AttributeError:
        cooling = None
        
    try:
        laundry = None
        laundry_icon = root.find('i', {'class': 'sc-qPLKk fonsds zsg-icon-laundry'})
        laundry_type = laundry_icon.find_next_sibling('span', {'class': 'Text-c11n-8-33-0__aiai24-0 sc-pAyMl eWnzUA'}).get_text()
        laundry = 1 if laundry_type == 'In Unit' else 0
    except AttributeError:
        laundry = None   
           
    try:
        description = root.find('div', {'class': 'ds-overview-section'}).get_text()
    except AttributeError:
        description = None    

    walk_score = None
    tran_score = None
    walk_score, tran_score = parse_scores(address_st)
    
    result = [[low_price, address_st, property_type, num_bed, num_bath, area, year_built, walk_score, tran_score, 
              parking_price, parking_type, cooling, laundry, description]]
    return result;
    


df = parse_zillow_search_page(get_url(1))
for i in range(2, 21):
    print(i)
    df = df.append(parse_zillow_search_page(get_url(i)))
df.to_csv('zillow_test.csv', encoding='utf-8', index=False)



# parse_zillow_house_page('https://www.zillow.com/homedetails/5541-Beeler-St-Pittsburgh-PA-15217/11629387_zpid/')
# print(parse_zillow_apartment_page('https://www.zillow.com/b/914-western-avenue-wt-914-4-pittsburgh-pa-5fwkYK/'))
# parse_scores('10428 Lindberg Ave, Pittsburgh, PA 15235')

In [134]:
addr = '''902 PJ McArdle Rd, Pittsburgh, PA 15203'''

addr = addr.replace('\n', ' ')
addr = addr.replace('CtE', 'Ct')
# addr = addr.replace('CtE', 'Ct')
usaddress.tag(addr)[0]
# normalize_address_record(addr)

OrderedDict([('AddressNumber', '902'),
             ('StreetName', 'PJ McArdle'),
             ('StreetNamePostType', 'Rd'),
             ('PlaceName', 'Pittsburgh'),
             ('StateName', 'PA'),
             ('ZipCode', '15203')])

In [3]:
def bool_object_to_float(input_value):
    if input_value == '':
        return 0.0
    elif input_value is True:
        return 1.0
    else:
        return 0.0
    
def numerical_object_to_float(input_value):
    if isinstance(input_value, float) and np.isnan(input_value):
        return np.nan
    else:
        return float(input_value)
    
def median_without_nan(df):
    return df.median(skipna=True, axis=0, numeric_only=True)

In [8]:
x = np.nan
print(np.nan == None)

False


In [10]:
def load_file():
    zillow_file = 'zillow.csv'
    ap_file = 'apartments.csv'
    apf_file = 'apartmentfinder.csv'
    
    df_zillow = pd.read_csv(zillow_file, header=0, sep=',', quotechar='"', dtype=object, keep_default_na=True)
    df_ap = pd.read_csv(ap_file, header=0, sep=',', quotechar='"', dtype=object, keep_default_na=True)
    df_apf = pd.read_csv(apf_file, header=0, sep=',', quotechar='"', dtype=object, keep_default_na=True)
    
    df_ap.drop_duplicates(subset=['Address', 'Price', 'Beds', 'Baths', 'Sqft'], inplace=True, ignore_index=True)
    
    df_ap['Description'] = df_ap['Description'] + df_ap['Amenities']
    df_ap.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Amenities'], inplace=True)
    
    return df_zillow, df_ap, df_apf

def format_address(df):

    # some patches in address name in order to use usaddress parser
    df['AddressFormatted'] = df['Address'].apply(lambda x: x.replace('\n', ' '))
    df['AddressFormatted'] = df['AddressFormatted'].apply(lambda x: x.replace('CtE', 'Ct'))
    df['AddressFormatted'] = df['AddressFormatted'].apply(lambda x: x.replace('249 Ave', '249 A Ave'))
    df['AddressFormatted'] = df['AddressFormatted'].apply(lambda x: x.replace('Roadway', 'Rd'))
    df['AddressFormatted'] = df['AddressFormatted'].apply(lambda x: x.split('–', 1)[0])
    # parse
    df['AddressFormatted'] = df['AddressFormatted'].apply(lambda x: usaddress.tag(x)[0]['AddressNumber'] + ' ' + usaddress.tag(x)[0]['StreetName'] + ' ' + 
                                                          usaddress.tag(x)[0]['StreetNamePostType'] + ' ' + usaddress.tag(x)[0]['PlaceName'] + ' ' + 
                                                          usaddress.tag(x)[0]['StateName'] + ' ' + usaddress.tag(x)[0]['ZipCode'])
    return df

def count_na(df):
    df['NACount'] = df.isna().sum(axis=1)
    return df

def merge_df(df_zillow, df_ap, df_apf):
    df_zillow = format_address(df_zillow)
    df_ap = format_address(df_ap)
    df_apf = format_address(df_apf)
    
    df_zillow = count_na(df_zillow)
    df_ap = count_na(df_ap)
    df_apf = count_na(df_apf)
    
    df = df_zillow.copy()
    
    while not df_apf.empty:
        new_address = df_apf.iloc[0]['AddressFormatted']
        if new_address in df['AddressFormatted'].values:
            old_na_count = df[df['AddressFormatted'] == new_address].iloc[0]['NACount']
            new_na_count = df_apf.iloc[0]['NACount']
            if new_na_count < old_na_count:
                df = df[~(df['AddressFormatted'] == new_address)]
                df = df.append(df_apf[df_apf['AddressFormatted'] == new_address])
        df_apf = df_apf[df_apf['AddressFormatted'] != new_address]
        
    while not df_ap.empty:
        new_address = df_ap.iloc[0]['AddressFormatted']
        if new_address in df['AddressFormatted'].values:
            old_na_count = df[df['AddressFormatted'] == new_address].iloc[0]['NACount']
            new_na_count = df_ap.iloc[0]['NACount']
            if new_na_count < old_na_count:
                df = df[~(df['AddressFormatted'] == new_address)]
                df = df.append(df_ap[df_ap['AddressFormatted'] == new_address])
        df_ap = df_ap[df_ap['AddressFormatted'] != new_address]
    
    
    df.sort_values(by=['AddressFormatted'], inplace=True, ignore_index=True)
    return df

# re-formatting for later analysis
def format_df(df):
    df['Cooling'] = df['Cooling'].apply(lambda x: True if x == '1' or x == 'True' or x is True else False)
    
    df['Price'] = df['Price'].apply(lambda x: numerical_object_to_float(x))
    df['PropertyType'] = df['PropertyType'].apply(lambda x: numerical_object_to_float(x))
    
    df['Beds'] = df['Beds'].apply(lambda x: numerical_object_to_float(x))
    df['Baths'] = df['Baths'].apply(lambda x: numerical_object_to_float(x))
    
    df['Sqft'] = df['Sqft'].apply(lambda x: numerical_object_to_float(x))
    df['YearBuilt'] = df['YearBuilt'].apply(lambda x: numerical_object_to_float(x))
    df['WalkScore'] = df['WalkScore'].apply(lambda x: numerical_object_to_float(x))
    df['TransitScore'] = df['TransitScore'].apply(lambda x: numerical_object_to_float(x))
    df['ParkingPrice'] = df['ParkingPrice'].apply(lambda x: numerical_object_to_float(x))
    df['ParkingType'] = df['ParkingType'].apply(lambda x: numerical_object_to_float(x))
    df['ParkingPrice'] = df['ParkingPrice'].apply(lambda x: numerical_object_to_float(x))
    df['Cooling'] = df['Cooling'].apply(lambda x: bool_object_to_float(x))
    df['Laundry'] = df['Laundry'].apply(lambda x: numerical_object_to_float(x))
    df['Address'] = df['AddressFormatted']
    
    df = df.drop(columns=['AddressFormatted', 'NACount'])
    
#     df = df[(df['Sqft'] > 100.0) | (df['Sqft'] < 0.0)]
    
    
    # assign median to missing values for certain columns
#     df_median = median_without_nan(df)
    return df

def split_fill_df(df):
    # dropna for bed and bath
    df = df.dropna(subset=['Beds', 'Baths'], axis=0)
    
    df_y = df['Price']
    df = df.drop(columns=['Price'])
    df_train, df_test, y_train, y_test = train_test_split(df, df_y, test_size=0.1, random_state=15388)
    
    df_train = df_train.join(y_train)
    df_test = df_test.join(y_test)
    
    df_train_median = median_without_nan(df_train)
    df_test_median = median_without_nan(df_test)
    
    print(df_train_median)
    
    df_train['YearBuilt'] = df_train['YearBuilt'].apply(lambda x: df_train_median['YearBuilt'] if np.isnan(x) else x)
    df_train['Sqft'] = df_train['Sqft'].apply(lambda x: df_train_median['Sqft'] if np.isnan(x) else x)
    df_train['WalkScore'] = df_train['WalkScore'].apply(lambda x: df_train_median['WalkScore'] if np.isnan(x) else x)
    df_train['TransitScore'] = df_train['TransitScore'].apply(lambda x: df_train_median['TransitScore'] if np.isnan(x) else x)
    # cooling is already filled
    df_train['Laundry'] = df_train['Laundry'].apply(lambda x: 0.0 if np.isnan(x) else x)
    df_train['ParkingPrice'] = df_train['ParkingPrice'].apply(lambda x: 0.0 if np.isnan(x) else x)
    df_train['ParkingType'] = df_train['ParkingType'].apply(lambda x: 0.0 if np.isnan(x) else x)
    
    df_test['YearBuilt'] = df_test['YearBuilt'].apply(lambda x: df_test_median['YearBuilt'] if np.isnan(x) else x)
    df_test['Sqft'] = df_test['Sqft'].apply(lambda x: df_test_median['Sqft'] if np.isnan(x) else x)
    df_test['WalkScore'] = df_test['WalkScore'].apply(lambda x: df_test_median['WalkScore'] if np.isnan(x) else x)
    df_test['TransitScore'] = df_test['TransitScore'].apply(lambda x: df_test_median['TransitScore'] if np.isnan(x) else x)
    # cooling is already filled
    df_test['Laundry'] = df_test['Laundry'].apply(lambda x: 0.0 if np.isnan(x) else x)
    df_test['ParkingPrice'] = df_test['ParkingPrice'].apply(lambda x: 0.0 if np.isnan(x) else x)
    df_test['ParkingType'] = df_test['ParkingType'].apply(lambda x: 0.0 if np.isnan(x) else x)
    
    
    return df_train, df_test


df_zillow, df_ap, df_apf = load_file()
df = merge_df(df_zillow, df_ap, df_apf)
df = format_df(df)
df_train, df_test = split_fill_df(df)

df_train.to_csv('all_train.csv', encoding='utf-8', index=False)
df_test.to_csv('all_test.csv', encoding='utf-8', index=False)

PropertyType       0.0
Beds               2.0
Baths              1.0
Sqft             886.0
YearBuilt       1950.0
WalkScore         83.0
TransitScore      64.0
ParkingPrice       0.0
ParkingType        1.0
Cooling            1.0
Laundry            1.0
Price           1266.5
dtype: float64
